# User interface Node #
This node will create a user interface for the Bug0 robot, in it the user will be able to set a new goal, delete the current one or get infromations about the number of goal reached or deleted.

### Libraries and global variables: ###
In this first cell a series of libraries needed for the User interface are imported and the global variables are istantiated.

In [ ]:
#! /usr/bin/env python3

%matplotlib widget
import rospy
import time
import sys
import select
import os
import tf
import actionlib
import actionlib.msg
import assignment_2_2022.msg
from nav_msgs.msg import Odometry
from sensor_msgs.msg import LaserScan
from assignment_2_2022.srv import Ngoal
import ipywidgets as widgets
from ipywidgets import VBox, HBox, Layout
import logging
from IPython.display import display
import jupyros as jr
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

act_clnt = None # istantiate the action client 
goal = assignment_2_2022.msg.PlanningGoal() # istantiate the goal message to send 
goal.target_pose.pose.position.z = 0.0 # set its third component equal to 0
info = Ngoal() # istantiate the info request for the service (number of goal reached and deleted)
srv_info = None # istantiate the responce
gf = 0 # goal flag to increase the number of goal only the first time the state pass to succeeded
count_scan = 0 # counter used to optimize the scan callback

### Visualizer class: ###
This class is used to create and then plot a graph in which represent the world.
first the `__init__` function initialize the figure and its graph, then the arrays in which the position of the robot and the goals will be stored are initialized, both for the single components 'x' and 'y' and for the one containing both like `ln_r` and `ln_g`, after that the title is set as the label of the axes, the legend and the counter used in the `odom_cb` to plot one position of the robot over ten.
secondly in the `plot_init`the limit for both the axes is set.
after that the `odom_cb` is defined, in it the method will check if it's the tenth time the method is called, in this case the position of the robot will be stored in the arrays `x_data` and `y_data` and the counter `odom_count` will be resetted, in all the cases the counter will be increased by one.
the `goal_update` function follow the same reasoning but will store the goal location under in the arrays `x_goal` and `y_goal` all the times the function is called.
finally the `refresh_plot` will be used to update the plot setting the data of the two lists `ln_r` for the position of the robot and `ln_g` for the one of the goal.

In [ ]:
class Visualizer:

    def __init__(self): # method to initialize the figure
        self.fig, self.ax = plt.subplots() # init the figure and the graph
        self.x_data, self.y_data = [], [] # init the arrays to store the robot position
        self.x_goal, self.y_goal = [], [] # init the arrays to store the goal positions
        self.ln_r, = plt.plot([], [], label = 'robot position', color = 'blue') # init the array in which store the final positions of the robot, with the line style
        self.ln_g, = plt.plot([], [], marker = 'X', label = 'goals', color = 'red', linestyle = 'None') # init the array in which store the final positions of the goals, with the markers style
        self.ax.set_title('World Representation') # set the graph title
        self.ax.set_xlabel('x') # set the x axis label
        self.ax.set_ylabel('y') # set the y axis label
        self.ax.legend() # add a legend
        self.odom_count = 0 # initialize the counter to 0
        
    def plot_init(self): # method to initilize the graph
        self.ax.set_xlim(-10, 10) # set the x axis bounds
        self.ax.set_ylim(-10, 10) # set the y axis bounds
        return self.ln_r, self.ln_g
     
    def odom_cb(self, msg): # callback of the /odom subscription
        if (self.odom_count == 10): # if it's the tenth time the method is called
            self.x_data.append(round(msg.pose.pose.position.x, 2)) # add the new robot x position to the array
            self.y_data.append(round(msg.pose.pose.position.y, 2)) # add the new robot y position to the array
            
            self.odom_count = 0 # reset the counter
        
        self.odom_count += 1 # in all the case increase the counter
            
    def goal_update(self, ngoal_x, ngoal_y): # method to store the goals position
        self.x_goal.append(ngoal_x) # add the x coordinate of the goal to the array
        self.y_goal.append(ngoal_y)# add the y coordinate of the goal to the array

    def refresh_plot(self, frame): # method to update the graph 
        self.ln_r.set_data(self.x_data, self.y_data) # set the robot position data to plot to the new ones
        self.ln_g.set_data(self.x_goal, self.y_goal) # set the goals position data to plot to the new ones
        return self.ln_r, self.ln_g

### scan_cb function: ###
This is the callback of the `/scan` subscriber, each time a new scan detection is acquired this method will be called, in detail one time over fifty the distance to the closer obstacle and the angle to it will be displayed in the relative text box. (note the Lidar of the robot scan the space in front of the robot in 720 partitions, the formula used below move the result in degree between -90 and 90)

In [ ]:
def scan_cb(msg):
    global count_scan, gf
    
    if (count_scan == 50): # one time over fifty
        Cod.value = round(min(msg.ranges), 2) # set the distance to the closer obstacle in the text box
        Coa.value = ((msg.ranges.index(min(msg.ranges)) + 1) / 4) - 90 # set the angle to it
        count_scan = -1 # reset the counter
    
    count_scan += 1 # increase the counter
    
    # keep track if the robot reach the goal:
    if (act_clnt.get_state() == 3 and gf == 0): # if i've reached the goal
        n_goal = rospy.get_param('n_goal') # get the actual number of goal
        n_goal = n_goal + 1 # increase it
        rospy.set_param('n_goal', n_goal) # load it in the paramserver
        gf = 1
    elif (act_clnt.get_state() != 3):
        gf = 0

### Node cell: ###
This cell will create the actual node, initially it will initialize it and will call the visualizer class under the name `vis` then will create an action client and wait for its existence, done that the node will subscribe to the `/odom` topic, to get the position of the robot, and the `/scan` topic to get the Lidar data.
Done that it will connect as service client to the service `/info` which will return the number of goal reached and deleted, wait for its existence and then initialize the parameters of the number of goals reached and deleted to zero.

In [ ]:
rospy.init_node('user_interface', anonymous=True) # init the node
vis = Visualizer() # istantiate the visualizer
act_clnt = actionlib.SimpleActionClient('/reaching_goal', assignment_2_2022.msg.PlanningAction) # connect to the action server
act_clnt.wait_for_server() # wait for the action server existence
sub_odom = jr.subscribe('/odom', Odometry, vis.odom_cb) # subscribe to the /odom topic
sud_scan = jr.subscribe('/scan',LaserScan, scan_cb) # subscribe to the /scan topic
srv_info = rospy.ServiceProxy("/info", Ngoal) # connect to the service server
rospy.wait_for_service("/info") # wait for the service server existance
rospy.set_param('n_goal', 0) # init the param n_goal (reached)
rospy.set_param('n_deleted', 0) # init the param n_deleted

### D_cb function: ###
This is the callback of the `D` button, used to delete the current goal, once the button is pressed it will check if there is an active goal, will delete it, will recover the actual number of goal deleted from the ros parameter server, increase it and then set it back in the ros parameter server.

In [ ]:
def D_cb(b):
    if ((act_clnt.get_state()) == 1): # in this case there is an active goal
        act_clnt.cancel_goal() # delete the goal
        n_deleted = rospy.get_param('n_deleted') # get the actual number of goal deleted
        n_deleted = n_deleted + 1 # increase it
        rospy.set_param('n_deleted', n_deleted) # load it in the paramserver

### N_cb function: ###
This is the callback of the `N` button, used to set a new goal, once pressed it will first check that the user finished to insert the previous goal, done that it activate the texts boxes to put the coordinate of the goal and deactivate itself.  

In [ ]:
def N_cb(b):
    if ((Gx.disabled == True) and (Gy.disabled == True)): # check if the previous goal insertion is finished
        Gx.disabled = False # unlock the x goal text box
        Gy.disabled = False # unlock the y goal text box
        N.disabled = True # lock the N button till the insertion of the goal is finshed

### I_cb function: ###
This is the callback of the `I` button, used to obtain information about the number of goal reached and deleted, it will first call the service `srv_info`, provided by an external node, obtained the data from it it will set the two value of the text boxes with the relative value.

In [ ]:
def I_cb(b):
    info = srv_info() # obtain the number of goal reached and deleted
    Gr.value = info.n_goal # set the number of goal reached in the right text box
    Gd.value = info.n_deleted # set the number of goal deleted in the right text box

### NGx_cb function: ###
This is the callback of the text box in which the x coordinate of the goal will be set, it will be called once the user press enter or once written something different from the previous value it click away from it.
the function will first disable the text box, meaning the user will no longer be able to type anything in it, then it will set the x coordinate in the goal message that then will be send to the action server.
done that the function will check if the user as putted also the other coordinate, and if so it will enable again the `N` button, send the goal, moreover it will call the visualizer method `vis.goal_update` to display the new goal on the graph passing its coordinates.

In [ ]:
def NGx_cb(value):   
    Gx.disabled = True # disable text box
    goal.target_pose.pose.position.x = float(Gx.value) # set the x coordinates of the goal
    if (Gy.disabled): # if the user has already set the other coordinate
        N.disabled = False # enable again the N button
        act_clnt.send_goal(goal) # send the goal
        vis.goal_update(Gx.value, Gy.value) # pass the coordinates of the goal to the visualizer

### NGy_cb function: ###
This is the callback of the text box in which the y coordinate of the goal will be set, it will work exactly as the `NGy_cb` function.

In [ ]:
def NGy_cb(value):    
    Gy.disabled = True # disable text box
    goal.target_pose.pose.position.y = float(Gy.value) # set the x coordinates of the goal
    if (Gx.disabled): # if the user has already set the other coordinate
        N.disabled = False # enable again the N button
        act_clnt.send_goal(goal) # send the goal
        vis.goal_update(Gx.value, Gy.value) # pass the coordinates of the goal to the visualizer

### User interface: ###
This cell will create the user interface that the user will use to control the behavior of the robot, in detail it will display three buttons, one for deleting the goal `D`, one for set a new goal `N` and the relative text boxes with the title to insert the coordinates and one last button `I` to recover the number of goal reached and deleted and display those information in two text boxes with the relative labels.
More over this cell will display the world representation, thanks to the `FuncAnimation` function and the `Visualizer` class, display the above mentioned user interface and create the callback for the buttons and text boxes.

In [ ]:
# different layout:
L1 = Layout(width = 'auto', height = '10%', min_width = '80px', min_height = '54px', max_width = '160px') 
L2 = Layout(width = 'auto', height = '5%', min_width = '40px', min_height = '20px', max_width = '80px')
L3 = Layout(width = 'auto', height = '5%', min_width = '80px', min_height = '20px', max_width = '160px')

# delete goal button:
D = widgets.Button(value = False, description = 'Delete Goal', disabled = False, button_style = 'danger', layout = L1, continuous_update = False)

# new goal button:
N = widgets.Button(value = False, description = 'New Goal', disabled = False, button_style = 'success', layout = L1, continuous_update = False)

# information button:
I = widgets.Button(value = False, description = 'Information', disabled = False, button_style = 'info', layout = L1, continuous_update = False)


# info box for the goal coordinates:
# name:
Gx_n = widgets.Text(value = 'x_goal', disabled = True, layout = L2)
Gy_n = widgets.Text(value = 'y_goal', disabled = True, layout = L2)
# data:
Gx = widgets.FloatText(value = '0.00', disabled = True, layout = L2, continuous_update = False)
Gy = widgets.FloatText(value = '0.00', disabled = True, layout = L2, continuous_update = False)
# combination:
Gx_c = VBox([Gx_n, Gx])
Gy_c = VBox([Gy_n, Gy])
Gxy = HBox([Gx_c, Gy_c])

# info box for the goal reached and deleted:
# name:
Gr_n = widgets.Text(value = 'goal_r', disabled = True, layout = L2)
Gd_n = widgets.Text(value = 'goal_d', disabled = True, layout = L2)
# data:
Gr = widgets.FloatText(value = '0.00', disabled = True, layout = L2, continuous_update = False)
Gd = widgets.FloatText(value = '0.00', disabled = True, layout = L2, continuous_update = False)
# combination:
Gr_c = VBox([Gr_n, Gr])
Gd_c = VBox([Gd_n, Gd])
Grd = HBox([Gr_c, Gd_c])

# combine everything
UI = VBox([D, HBox([N, Gxy]), HBox([I, Grd])])

# info box for the closer obstacle:
# name:
Co_n = widgets.Text(value = 'closer obstacle', disabled = True, layout = L3)
Cod_n = widgets.Text(value = 'dist', disabled = True, layout = L2)
Coa_n = widgets.Text(value = 'angle', disabled = True, layout = L2)
# data:
Cod = widgets.FloatText(value = '0.00', disabled = True, layout = L2, continuous_update = True)
Coa = widgets.FloatText(value = '0.00', disabled = True, layout = L2, continuous_update = True)
# combination:
Cod_c = VBox([Cod_n, Cod])
Coa_c = VBox([Coa_n, Coa])
Coda_c = HBox([Cod_c, Coa_c])
Coda = VBox([Co_n, Coda_c])

# combine UI and IM and print:
CUI = HBox([UI, Coda])

# plot:
ani = FuncAnimation(vis.fig, vis.refresh_plot, init_func=vis.plot_init, cache_frame_data=False)
plt.show(block = True)

display(CUI)

# callbacks:
D.on_click(D_cb) 
N.on_click(N_cb)
Gx.observe(NGx_cb, names = 'value')
Gy.observe(NGy_cb, names = 'value')
I.on_click(I_cb)